# Derivation of euler equations in primitive form
Derivation of the Weiss and Smith preconditioning using the simplification of Economon.
We are following the 1D derivation of maxima_weiss_smith_1D here and extend it to 2D

In [1]:
kill(all);

(%o0)                                done

# Analysis of the Euler Equations
## 1. conservative form


    

We want to solve the PDE:

$$ \frac{\partial U}{\partial t} + \frac{\partial F(U)}{\partial x} + \frac{\partial G(U)}{\partial y} = 0$$


    
For the 2D problem we have the following definitions:


$$
U = \left\{
    \begin{array}\\
        \rho               &  \\
        \rho u        &  \\
        \rho v        &  \\
        \rho c_p T & 
    \end{array}
\right\}
$$


$$
F = \left\{
    \begin{array}\\
        \rho  u                  &  \\
        p+\rho u^2               &  \\
        \rho  u  v          &  \\
        (\rho c_p T ) u & 
    \end{array}
\right\}
$$


$$
G = \left\{
    \begin{array}\\
        \rho  v          &  \\
        \rho u v & \\
        p+\rho v^2               &  \\
        (\rho c_p T ) v & 
    \end{array}
\right\}
$$

Note that we have neglected the pressure term in the convective flux of the Euler equations

In [2]:
consSystem : [U1=rho,U2=rho*u, U3=rho*v, U4=rho*E];

F:ratsimp((matrix([rho*u,rho*u*u+p,rho*u*v,u*(rho*E) ])));
    
G:ratsimp((matrix([rho*v,rho*u*v,rho*v*v+p,v*(rho*E) ])));

(%o1)           [U1 = rho, U2 = rho u, U3 = rho v, U4 = E rho]

(%o2)               [             2                       ]
                    [ rho u  rho u  + p  rho u v  E rho u ]

(%o3)               [                      2              ]
                    [ rho v  rho u v  rho v  + p  E rho v ]

The internal energy follows from the relationships : 
    
$$E = H - \frac{p}{\rho} = e + q$$
$$H = h + q = c_pT + q$$
    
$$E = c_pT + q - \frac{p}{\rho}$$
    

    
The pressure follows from the pressure definition: $p = \rho R T$ and $R = c_p-c_v = c_p \frac{\gamma-1}{\gamma}$
    


In [3]:
E : cp*T;

(%o4)                                T cp

In [4]:
p : rho*R*T;

(%o5)                               R T rho

In [5]:
R : cp-cv;
/* gamma = cp/cv */
cv : cp/gamma;

R : ev(R,nouns);

factor(ratsimp(ev(p,nouns)));

(%o6)                               cp - cv

                                      cp
(%o7)                                -----
                                     gamma

                                        cp
(%o8)                             cp - -----
                                       gamma

                             T cp rho (gamma - 1)
(%o9)                        --------------------
                                    gamma

    
We now map the conservative variables to $(\rho,u,v,T)$
    


# 4. Preconditioning matrix $\Gamma$

The above shows how the conversion from conservative to primitive variables work. Note that in the paper of Weiss and Smith, the derivatives of density with respect to temperature and pressure are kept.

Note that $P=\rho R T$, and $\rho=\frac{P}{RT}$, so the derivative is:

$$ \frac{\partial \rho}{\partial p} = \frac{1}{RT} $$

And since $c^2 = \gamma R T$, we have:

$$ \frac{\partial \rho}{\partial p} = \frac{\gamma}{c^2} $$

For derivative wrt temperarure we have :
    


In [6]:
kill(p);
subst(p=rho*R*T,diff(p/(R*T),T));
diff(p/(R*T),p);

(%o10)                               done

                                       rho
(%o11)                               - ---
                                        T

                                      1
(%o12)                          --------------
                                         cp
                                T (cp - -----)
                                        gamma

So we have:
$$ \frac{\partial \rho}{\partial T} = -\frac{\rho}{T} $$
$$ \frac{\partial \rho}{\partial p} = \frac{1}{RT} $$

We can substitute these into the matrix M above to get the result from Weiss and Smith:
    


In [7]:
depends(rho,[p,T]);

(%o13)                            [rho(p, T)]


Now we compute the transformation matrix $M = \frac{\partial U}{\partial V}$, equation (7) in the paper of Economon, or below eq. 2 in Weiss and Smith:


In [8]:
U:matrix([rho,rho*u,rho*v,rho*E]);
V:matrix([p,u,v,T]);

/* *we directly convert from conservative variables to primitive variables */
dUdV : ratsimp(jacobian(args(U)[1],args(V)[1]));

(%o14)                  [ rho  rho u  rho v  T cp rho ]

(%o15)                          [ p  u  v  T ]

                  [   drho                      drho        ]
                  [   ----      0    0          ----        ]
                  [    dp                        dT         ]
                  [                                         ]
                  [  drho                      drho         ]
                  [  ---- u    rho   0         ---- u       ]
                  [   dp                        dT          ]
(%o16)            [                                         ]
                  [  drho                      drho         ]
                  [  ---- v     0   rho        ---- v       ]
                  [   dp                        dT          ]
                  [                                         ]
                  [      drho                 drho          ]
                  [ T cp ----   0    0   T cp ---- + cp rho ]
                  [       dp                   dT           ]

We see that we have recovered the result of Economon.

Note that $p=\rho R T$, and $\frac{a^2}{\gamma} = \frac{p}{\rho}$, so:
$$RT = \frac{a^2}{\gamma}$$

and with $\frac{c_p}{c_v}=\gamma$ and  $R = c_p - c_v = c_p(1-\frac{1}{\gamma})$:
$$T = \frac{a^2}{\gamma}\frac{1}{c_p(1-\frac{1}{\gamma})},$$
and finally:

$$ T = \frac{a^2}{ c_p (\gamma - 1)}$$

In [9]:
dUdV2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=1/(R*T)],dUdV);
dUdV2 : ratsimp(subst([T=c*c/(cp*(gamma-1))],dUdV2));

                     [       1                      rho  ]
                     [ --------------   0    0    - ---  ]
                     [          cp                   T   ]
                     [ T (cp - -----)                    ]
                     [         gamma                     ]
                     [                                   ]
                     [       u                     rho u ]
                     [ --------------  rho   0   - ----- ]
                     [          cp                   T   ]
                     [ T (cp - -----)                    ]
                     [         gamma                     ]
(%o17)               [                                   ]
                     [       v                     rho v ]
                     [ --------------   0   rho  - ----- ]
                     [          cp                   T   ]
                     [ T (cp - -----)                    ]
                     [         gamma                    

             [   gamma                  cp rho gamma - cp rho   ]
             [   -----     0    0     - ---------------------   ]
             [     2                              2             ]
             [    c                              c              ]
             [                                                  ]
             [  u gamma               cp rho u gamma - cp rho u ]
             [  -------   rho   0   - ------------------------- ]
             [     2                              2             ]
(%o18)       [    c                              c              ]
             [                                                  ]
             [  v gamma               cp rho v gamma - cp rho v ]
             [  -------    0   rho  - ------------------------- ]
             [     2                              2             ]
             [    c                              c              ]
             [                                                  ]
          

Note that the flux is $\frac{\partial F(V)}{\partial x}$, with $V=(p,u,v,T)$ and because we have defined the derivative of density with respect to pressure and temperature, we can obtain the transformation into quasilinear form directly using:
$$\frac{\partial F(V)}{\partial x} = \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}$$

In [11]:
ev(F,nouns);
ev(G,nouns);

(%o20)            [             2                          ]
                  [ rho u  rho u  + p  rho u v  T cp rho u ]

(%o21)            [                      2                 ]
                  [ rho v  rho u v  rho v  + p  T cp rho v ]

and its Jacobian with respect to $V=(\rho, u,v, T)$ can be determined directly (because we also have the derivative of H(T) defined):

In [12]:
V;

(%o22)                          [ p  u  v  T ]

In [13]:
dFdV : ratsimp(ev(jacobian(args(F)[1],args(V)[1]),nouns));
dGdV : ratsimp(ev(jacobian(args(G)[1],args(V)[1]),nouns));

           [   drho                                drho           ]
           [   ---- u       rho       0            ---- u         ]
           [    dp                                  dT            ]
           [                                                      ]
           [ drho  2                              drho  2         ]
           [ ---- u  + 1  2 rho u     0           ---- u          ]
           [  dp                                   dT             ]
(%o23)     [                                                      ]
           [  drho                                drho            ]
           [  ---- u v     rho v    rho u         ---- u v        ]
           [   dp                                  dT             ]
           [                                                      ]
           [      drho                           drho             ]
           [ T cp ---- u  T cp rho    0    (T cp ---- + cp rho) u ]
           [       dp                           

           [   drho                                drho           ]
           [   ---- v       0      rho             ---- v         ]
           [    dp                                  dT            ]
           [                                                      ]
           [  drho                                drho            ]
           [  ---- u v    rho v   rho u           ---- u v        ]
           [   dp                                  dT             ]
(%o24)     [                                                      ]
           [ drho  2                              drho  2         ]
           [ ---- v  + 1    0    2 rho v          ---- v          ]
           [  dp                                   dT             ]
           [                                                      ]
           [      drho                           drho             ]
           [ T cp ---- v    0    T cp rho  (T cp ---- + cp rho) v ]
           [       dp                           

In [14]:
dFdV2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=1/(R*T)],dFdV);
dFdV2 : ratsimp(subst([T=c*c/(cp*(gamma-1))],dFdV2));
dGdV2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=1/(R*T)],dGdV);
dGdV2 : ratsimp(subst([T=c*c/(cp*(gamma-1))],dGdV2));

              [         u                               rho u  ]
              [   --------------      rho       0     - -----  ]
              [            cp                             T    ]
              [   T (cp - -----)                               ]
              [           gamma                                ]
              [                                                ]
              [        2                                    2  ]
              [       u                                rho u   ]
              [ -------------- + 1  2 rho u     0    - ------  ]
              [          cp                              T     ]
              [ T (cp - -----)                                 ]
(%o25)        [         gamma                                  ]
              [                                                ]
              [        u v                             rho u v ]
              [   --------------     rho v    rho u  - ------- ]
              [          

       [    u gamma                           cp rho u gamma - cp rho u   ]
       [    -------        rho       0      - -------------------------   ]
       [       2                                          2               ]
       [      c                                          c                ]
       [                                                                  ]
       [  2          2                               2                 2  ]
       [ u  gamma + c                        cp rho u  gamma - cp rho u   ]
       [ -------------   2 rho u     0     - ---------------------------  ]
       [       2                                          2               ]
(%o26) [      c                                          c                ]
       [                                                                  ]
       [   u v gamma                        cp rho u v gamma - cp rho u v ]
       [   ---------      rho v    rho u  - ----------------------------- ]
       [    

              [         v                               rho v  ]
              [   --------------      0      rho      - -----  ]
              [            cp                             T    ]
              [   T (cp - -----)                               ]
              [           gamma                                ]
              [                                                ]
              [        u v                             rho u v ]
              [   --------------    rho v   rho u    - ------- ]
              [            cp                             T    ]
              [   T (cp - -----)                               ]
              [           gamma                                ]
(%o27)        [                                                ]
              [        2                                    2  ]
              [       v                                rho v   ]
              [ -------------- + 1    0    2 rho v   - ------  ]
              [          

       [    v gamma                           cp rho v gamma - cp rho v   ]
       [    -------       0       rho       - -------------------------   ]
       [       2                                          2               ]
       [      c                                          c                ]
       [                                                                  ]
       [   u v gamma                        cp rho u v gamma - cp rho u v ]
       [   ---------    rho v    rho u    - ----------------------------- ]
       [       2                                          2               ]
       [      c                                          c                ]
(%o28) [                                                                  ]
       [  2          2                               2                 2  ]
       [ v  gamma + c                        cp rho v  gamma - cp rho v   ]
       [ -------------    0     2 rho v    - ---------------------------  ]
       [    

### We can now write the pde in primitive variables $V$ as:
$$\frac{\partial U}{\partial t} + \frac{\partial F}{\partial x}+ \frac{\partial G}{\partial y} =0$$

$$\frac{\partial U}{\partial V} \frac{\partial V}{\partial t} + \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}+ \frac{\partial G}{\partial V}\frac{\partial V}{\partial y}=0$$

$$ \frac{\partial V}{\partial t} + [\frac{\partial U}{\partial V}]^{-1} \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}+ [\frac{\partial U}{\partial V}]^{-1} \frac{\partial G}{\partial V}\frac{\partial V}{\partial y}=0$$


In [15]:
ratsimp(invert(dUdV2));

               [                                    gamma - 1 ]
               [            0              0    0   --------- ]
               [                                      gamma   ]
               [                                              ]
               [             u             1                  ]
               [          - ---           ---   0       0     ]
               [            rho           rho                 ]
(%o29)         [                                              ]
               [             v                  1             ]
               [          - ---            0   ---      0     ]
               [            rho                rho            ]
               [                                              ]
               [             2                                ]
               [            c                          1      ]
               [ - ---------------------   0    0    ------   ]
               [   cp rho gamma - cp rho

In [16]:
Ac : ratsimp(invert(dUdV2).dFdV2);

                             [       2           ]
                             [      c  rho       ]
                             [  u   ------  0  0 ]
                             [      gamma        ]
                             [                   ]
(%o30)                       [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
                             [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [  0     0     0  u ]

In [17]:
ratexpand(eigenvalues(Ac));

                         c            c
(%o31)        [[u - -----------, ----------- + u, u], [1, 1, 2]]
                    sqrt(gamma)  sqrt(gamma)

In [18]:
Bc : ratsimp(invert(dUdV2).dGdV2);
ratexpand(eigenvalues(Bc));

                             [          2        ]
                             [         c  rho    ]
                             [  v   0  ------  0 ]
                             [         gamma     ]
                             [                   ]
(%o32)                       [  0   v    0     0 ]
                             [                   ]
                             [  1                ]
                             [ ---  0    v     0 ]
                             [ rho               ]
                             [                   ]
                             [  0   0    0     v ]

                         c            c
(%o33)        [[v - -----------, ----------- + v, v], [1, 1, 2]]
                    sqrt(gamma)  sqrt(gamma)

We now the different eigenvalues of the quasilinear non-conservative system of Euler equations.

We have recovered the results of Economon (2018)

We replace the term $\frac{\partial \rho}{\partial p} = \frac{1}{b^2}$

In [19]:
Gamma : subst(diff(rho,p)=1/b^2,dUdV);
dFdV : subst(diff(rho,p)=1/b^2,dFdV);
dGdV : subst(diff(rho,p)=1/b^2,dGdV);

                    [  1                     drho        ]
                    [  --    0    0          ----        ]
                    [   2                     dT         ]
                    [  b                                 ]
                    [                                    ]
                    [  u                    drho         ]
                    [  --   rho   0         ---- u       ]
                    [   2                    dT          ]
                    [  b                                 ]
(%o34)              [                                    ]
                    [  v                    drho         ]
                    [  --    0   rho        ---- v       ]
                    [   2                    dT          ]
                    [  b                                 ]
                    [                                    ]
                    [ T cp                 drho          ]
                    [ ----   0    0   T cp ---- + cp rho

              [   u                              drho           ]
              [   --      rho       0            ---- u         ]
              [    2                              dT            ]
              [   b                                             ]
              [                                                 ]
              [  2                                              ]
              [ u                               drho  2         ]
              [ -- + 1  2 rho u     0           ---- u          ]
              [  2                               dT             ]
(%o35)        [ b                                               ]
              [                                                 ]
              [  u v                            drho            ]
              [  ---     rho v    rho u         ---- u v        ]
              [   2                              dT             ]
              [  b                                              ]
          

              [   v                              drho           ]
              [   --      0      rho             ---- v         ]
              [    2                              dT            ]
              [   b                                             ]
              [                                                 ]
              [  u v                            drho            ]
              [  ---    rho v   rho u           ---- u v        ]
              [   2                              dT             ]
              [  b                                              ]
(%o36)        [                                                 ]
              [  2                                              ]
              [ v                               drho  2         ]
              [ -- + 1    0    2 rho v          ---- v          ]
              [  2                               dT             ]
              [ b                                               ]
          

In [20]:
invGamma : ratsimp(invert(Gamma));

                  [    2 drho    2                   2 drho ]
                  [ T b  ---- + b  rho              b  ---- ]
                  [       dT                            dT  ]
                  [ ------------------   0    0   - ------- ]
                  [        rho                      cp rho  ]
                  [                                         ]
                  [          u           1                  ]
                  [       - ---         ---   0       0     ]
(%o37)            [         rho         rho                 ]
                  [                                         ]
                  [          v                1             ]
                  [       - ---          0   ---      0     ]
                  [         rho              rho            ]
                  [                                         ]
                  [          T                       1      ]
                  [       - ---          0    0    ------   ]
        

We can now construct the convective flux Jacobian $A_\Gamma^c$, this is use in the FDS numerical scheme in Economon's AIAA 2018 paper, eq. (28):

In [21]:
Ac : ratsimp(invGamma.dFdV);

                             [       2           ]
                             [  u   b  rho  0  0 ]
                             [                   ]
                             [  1                ]
(%o38)                       [ ---    u     0  0 ]
                             [ rho               ]
                             [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [  0     0     0  u ]

In [22]:
eigenvectors(Ac);

(%o39) [[[u - b, u + b, u], [1, 1, 2]], 
                1                   1
      [[[1, - -----, 0, 0]], [[1, -----, 0, 0]], [[0, 0, 1, 0], [0, 0, 0, 1]]]]
              b rho               b rho

In [24]:
lambdaA : ratsimp(eigenvalues(Ac));

(%o41)                  [[u - b, u + b, u], [1, 1, 2]]

### We see that when we replace all derivatives $\frac{\partial \rho}{\partial p}$ by $\frac{1}{b^2}$, the simple eigenvalues $(u,u-b,u+b)$ are obtained, as in the paper of Economon.

In [27]:
Bc : ratsimp(invGamma.dGdV);
eigenvectors(Bc);
lambdaB : ratsimp(eigenvalues(Bc));

                             [          2        ]
                             [  v   0  b  rho  0 ]
                             [                   ]
                             [  0   v    0     0 ]
(%o48)                       [                   ]
                             [  1                ]
                             [ ---  0    v     0 ]
                             [ rho               ]
                             [                   ]
                             [  0   0    0     v ]

(%o49) [[[v - b, v + b, v], [1, 1, 2]], 
                   1                   1
      [[[1, 0, - -----, 0]], [[1, 0, -----, 0]], [[0, 1, 0, 0], [0, 0, 0, 1]]]]
                 b rho               b rho

(%o50)                  [[v - b, v + b, v], [1, 1, 2]]

In [28]:
A : ratsimp(Ac*nx+Bc*ny);

            [                2            2                      ]
            [ ny v + nx u   b  nx rho    b  ny rho        0      ]
            [                                                    ]
            [     nx                                             ]
            [     ---      ny v + nx u       0            0      ]
(%o51)      [     rho                                            ]
            [                                                    ]
            [     ny                                             ]
            [     ---           0       ny v + nx u       0      ]
            [     rho                                            ]
            [                                                    ]
            [      0            0            0       ny v + nx u ]

In [29]:
lambda : eigenvalues(A);

                                2     2                          2     2
(%o52) [[ny v + nx u - b sqrt(ny  + nx ), ny v + nx u + b sqrt(ny  + nx ), 
                                                       ny v + nx u], [1, 1, 2]]

We now want to decompose the matrix into $A = P\cdot \overline{\overline\lambda} \cdot P^{-1}$

In [42]:
V : eigenvectors(A);
P : V[2];

                                 2     2                          2     2
(%o71) [[[ny v + nx u - b sqrt(ny  + nx ), ny v + nx u + b sqrt(ny  + nx ), 
                                            2     2                2     2
                                  nx sqrt(ny  + nx )     ny sqrt(ny  + nx )
ny v + nx u], [1, 1, 2]], [[[1, - -------------------, - -------------------, 
                                       2       2              2       2
                                  (b ny  + b nx ) rho    (b ny  + b nx ) rho
                    2     2              2     2
          nx sqrt(ny  + nx )   ny sqrt(ny  + nx )
0]], [[1, -------------------, -------------------, 0]], 
               2       2            2       2
          (b ny  + b nx ) rho  (b ny  + b nx ) rho
          nx
[[0, 1, - --, 0], [0, 0, 0, 1]]]]
          ny

                         2     2                2     2
               nx sqrt(ny  + nx )     ny sqrt(ny  + nx )
(%o72) [[[1, - -------------------, - -------------------, 0]], 
                    2       2              2       2
               (b ny  + b nx ) rho    (b ny  + b nx ) rho
               2     2              2     2
     nx sqrt(ny  + nx )   ny sqrt(ny  + nx )
[[1, -------------------, -------------------, 0]], 
          2       2            2       2
     (b ny  + b nx ) rho  (b ny  + b nx ) rho
          nx
[[0, 1, - --, 0], [0, 0, 0, 1]]]
          ny

In [49]:
ratsimp(P);

                         2     2                2     2
               nx sqrt(ny  + nx )     ny sqrt(ny  + nx )
(%o79) [[[1, - -------------------, - -------------------, 0]], 
                    2       2              2       2
               (b ny  + b nx ) rho    (b ny  + b nx ) rho
               2     2              2     2
     nx sqrt(ny  + nx )   ny sqrt(ny  + nx )
[[1, -------------------, -------------------, 0]], 
          2       2            2       2
     (b ny  + b nx ) rho  (b ny  + b nx ) rho
          nx
[[0, 1, - --, 0], [0, 0, 0, 1]]]
          ny

In [43]:
grind(lambda);

(%o73)                               done


[[ny*v+nx*u-b*sqrt(ny^2+nx^2),ny*v+nx*u+b*sqrt(ny^2+nx^2),ny*v+nx*u],[1,1,2]]$


In [62]:
lambda_s : matrix([lambda_1,0,0,0],[0,lambda_2,0,0],[0,0,lambda_3,0],[0,0,0,lambda_4]);
lambda : matrix([ny*v+nx*u-b*sqrt(ny^2+nx^2),0,0,0],[0,ny*v+nx*u+b*sqrt(ny^2+nx^2),0,0],[0,0,ny*v+nx*u,0],[0,0,0,ny*v+nx*u]);

                  [ lambda_1     0         0         0     ]
                  [                                        ]
                  [    0      lambda_2     0         0     ]
(%o91)            [                                        ]
                  [    0         0      lambda_3     0     ]
                  [                                        ]
                  [    0         0         0      lambda_4 ]

                [                        2     2  ]
                [ ny v + nx u - b sqrt(ny  + nx ) ]
                [                                 ]
(%o92)  Col 1 = [                0                ]
                [                                 ]
                [                0                ]
                [                                 ]
                [                0                ]
         [                0                ]         [      0      ]
         [                                 ]         [             ]
         [                        2     2  ]         [      0      ]
 Col 2 = [ ny v + nx u + b sqrt(ny  + nx ) ] Col 3 = [             ]
         [                                 ]         [ ny v + nx u ]
         [                0                ]         [             ]
         [                                 ]         [      0      ]
         [                0                ]
         [      0      ]
         [             ]
      

In [46]:
grind(P);

(%o75)                               done


[[[1,-(nx*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),
   -(ny*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),0]],
 [[1,(nx*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),
   (ny*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),0]],[[0,1,-nx/ny,0],[0,0,0,1]]]$


In [59]:
P : matrix([1,-(nx*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),
   -(ny*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),0],[1,(nx*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),
   (ny*sqrt(ny^2+nx^2))/((b*ny^2+b*nx^2)*rho),0],[0,1,-nx/ny,0],[0,0,0,1]);
   P : transpose(P);

            [                2     2                2     2      ]
            [      nx sqrt(ny  + nx )     ny sqrt(ny  + nx )     ]
            [ 1  - -------------------  - -------------------  0 ]
            [           2       2              2       2         ]
            [      (b ny  + b nx ) rho    (b ny  + b nx ) rho    ]
            [                                                    ]
            [               2     2                2     2       ]
            [     nx sqrt(ny  + nx )     ny sqrt(ny  + nx )      ]
(%o88)      [ 1   -------------------    -------------------   0 ]
            [          2       2              2       2          ]
            [     (b ny  + b nx ) rho    (b ny  + b nx ) rho     ]
            [                                                    ]
            [                                     nx             ]
            [ 0            1                    - --           0 ]
            [                                     ny          

            [           1                     1            0    0 ]
            [                                                     ]
            [             2     2              2     2            ]
            [   nx sqrt(ny  + nx )   nx sqrt(ny  + nx )           ]
            [ - -------------------  -------------------   1    0 ]
            [        2       2            2       2               ]
            [   (b ny  + b nx ) rho  (b ny  + b nx ) rho          ]
(%o89)      [                                                     ]
            [             2     2              2     2            ]
            [   ny sqrt(ny  + nx )   ny sqrt(ny  + nx )     nx    ]
            [ - -------------------  -------------------  - --  0 ]
            [        2       2            2       2         ny    ]
            [   (b ny  + b nx ) rho  (b ny  + b nx ) rho          ]
            [                                                     ]
            [           0                     0 

In [61]:
ratsimp(P.lambda.invert(P));

            [                2            2                      ]
            [ ny v + nx u   b  nx rho    b  ny rho        0      ]
            [                                                    ]
            [     nx                                             ]
            [     ---      ny v + nx u       0            0      ]
(%o90)      [     rho                                            ]
            [                                                    ]
            [     ny                                             ]
            [     ---           0       ny v + nx u       0      ]
            [     rho                                            ]
            [                                                    ]
            [      0            0            0       ny v + nx u ]

In [53]:
A;

            [                2            2                      ]
            [ ny v + nx u   b  nx rho    b  ny rho        0      ]
            [                                                    ]
            [     nx                                             ]
            [     ---      ny v + nx u       0            0      ]
(%o84)      [     rho                                            ]
            [                                                    ]
            [     ny                                             ]
            [     ---           0       ny v + nx u       0      ]
            [     rho                                            ]
            [                                                    ]
            [      0            0            0       ny v + nx u ]

In [63]:
ratsimp(P.lambda_s.invert(P));

                [           lambda_2 + lambda_1            ]
                [           -------------------            ]
                [                    2                     ]
                [                                          ]
                [                                 2     2  ]
                [ (lambda_2 - lambda_1) nx sqrt(ny  + nx ) ]
                [ ---------------------------------------- ]
                [                2         2               ]
(%o93)  Col 1 = [         (2 b ny  + 2 b nx ) rho          ]
                [                                          ]
                [                                 2     2  ]
                [ (lambda_2 - lambda_1) ny sqrt(ny  + nx ) ]
                [ ---------------------------------------- ]
                [                2         2               ]
                [         (2 b ny  + 2 b nx ) rho          ]
                [                                          ]
                [       

And we have retrieved the final scaling matrix for the Flux Difference Splitting

In [45]:
Gamma : subst(diff(rho,p)=theta,dWdQ);

                [                                 drho       ]
                [    theta       0      0         ----       ]
                [                                  dT        ]
                [                                            ]
                [                                drho        ]
                [   theta u     rho     0        ---- u      ]
                [                                 dT         ]
(%o83)          [                                            ]
                [                                drho        ]
                [   theta v      0     rho       ---- v      ]
                [                                 dT         ]
                [                                            ]
                [                              drho          ]
                [ H theta - 1  rho u  rho v  H ---- + cp rho ]
                [                               dT           ]

This is the convective matrix for the quasilinear system with primitive variables $V$.

In [46]:
inv_dUdV : ratsimp(invert(M));

                [             2    2           2    2           ]
                [           (v  + u ) gamma - v  - u            ]
                [           -------------------------           ]
                [                       2                       ]
                [                                               ]
                [                        u                      ]
                [                     - ---                     ]
                [                       rho                     ]
(%o84)  Col 1 = [                                               ]
                [                        v                      ]
                [                     - ---                     ]
                [                       rho                     ]
                [                                               ]
                [   2    2       2        2     2             2 ]
                [ (v  + u ) gamma  + ((- v ) - u ) gamma - 2 c  ]
          

In [47]:
A2:ratsimp(inv_dUdV.dFdV);

                   drho    2   drho  3            drho             drho    2
(%o85) matrix([- ((---- u v  + ---- u  + (2 - 2 H ----) u) gamma - ---- u v
                    dp          dp                 dp               dp
   drho  3        drho
 - ---- u  + (2 H ---- - 2) u)/2, 
    dp             dp
        2        2                         2        2
  (rho v  + rho u  - 2 H rho) gamma - rho v  - rho u  + 2 H rho
- -------------------------------------------------------------, 0, 
                                2
    drho    2   drho  3           drho                         drho    2
- ((---- u v  + ---- u  + ((- 2 H ----) - 2 cp rho) u) gamma - ---- u v
     dT          dT                dT                           dT
   drho  3        drho                      1
 - ---- u  + (2 H ---- + 2 cp rho) u)/2], [---, u, 0, 0], [0, 0, u, 0], 
    dT             dT                      rho
     drho    2   drho  3            drho          2
[- ((---- u v  + ---- u  + (2 - 2 H ----) u) 

In [48]:
A3:ratsimp(subst(diff(rho,T)=-rho/T,A2));
A3:ratsimp(subst(diff(rho,p)=gamma/c^2,A3));

                   drho    2   drho  3            drho             drho    2
(%o86) matrix([- ((---- u v  + ---- u  + (2 - 2 H ----) u) gamma - ---- u v
                    dp          dp                 dp               dp
   drho  3        drho
 - ---- u  + (2 H ---- - 2) u)/2, 
    dp             dp
        2        2                         2        2
  (rho v  + rho u  - 2 H rho) gamma - rho v  - rho u  + 2 H rho
- -------------------------------------------------------------, 0, 
                                2
         2        3                                        2        3
((rho u v  + rho u  + (2 T cp - 2 H) rho u) gamma - rho u v  - rho u
                                   1
 + (2 H - 2 T cp) rho u)/(2 T)], [---, u, 0, 0], [0, 0, u, 0], 
                                  rho
     drho    2   drho  3            drho          2
[- ((---- u v  + ---- u  + (2 - 2 H ----) u) gamma
      dp          dp                 dp
       drho    2    drho  3        drho               

                      2    3               2
(%o87) matrix([- ((u v  + u  - 2 H u) gamma
          2     3       2                      2        2
 + ((- u v ) - u  + (2 c  + 2 H) u) gamma - 2 c  u)/(2 c ), 
        2        2                         2        2
  (rho v  + rho u  - 2 H rho) gamma - rho v  - rho u  + 2 H rho
- -------------------------------------------------------------, 0, 
                                2
         2        3                                        2        3
((rho u v  + rho u  + (2 T cp - 2 H) rho u) gamma - rho u v  - rho u
                                   1
 + (2 H - 2 T cp) rho u)/(2 T)], [---, u, 0, 0], [0, 0, u, 0], 
                                  rho
       2    3               3          2     3       2                2
   (u v  + u  - 2 H u) gamma  + ((- u v ) - u  + (2 c  + 2 H) u) gamma
[- --------------------------------------------------------------------, 
                        2                   2
                     2 c  cp r

We also see that the matrix $\tilde A$ actually should also depend on the derivatives of the density:

In [49]:
Atilde;

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o88)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

The matrix M is the transformation matrix to go from conservative to primitive variables. We should now also go from conservative form to quasilinear (nonconservative) form. For this, we need to write F in terms of the primitive variables and find $A = \frac{\partial F}{\partial U}$

First, F in terms of the primitive variables:

In [50]:
Atilde2 : ratsimp(invert(M).A1.M);

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o89)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

with this form, we can still retrieve the eigenvalues in their most simplified form:

In [51]:
ratexpand(ratsimp(eigenvalues(Atilde2)));

(%o90)                  [[u - c, u + c, u], [1, 1, 2]]

Note that we can now get the preconditioning matrix by simply replacing $\frac{\partial \rho}{\partial p}$ by the preconditioning constant $\beta$. This is because this is the only term that determines the eigenvalues $u+c$ and $u-c$. So if we want to modify the condition number, we need to modify this term.


In [52]:
Mp : subst(diff(rho,p)=beta,M);

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o91)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

In [53]:
Mp : subst(diff(rho,T)=-rho/T,Mp);

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o92)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

In [54]:
Mp : subst(H = cp*T +q,Mp);

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o93)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

What are the eigenvalues of the new system:

In [55]:
Ap : ratsimp(invert(Mp).Atilde);
/*ratexpand(ratsimp(eigenvalues(Ap)));*/

                       2        3                       2        3
               (rho u v  + rho u  - 2 u) gamma - rho u v  - rho u  + 2 u
(%o94) matrix([---------------------------------------------------------, 
                                         2 rho
   2         2     2                 2      2           2         2
((c  cp rho v  + (c  cp rho - 2 cp) u  + 2 c ) gamma - c  cp rho v
            2          2      2
 + (2 cp - c  cp rho) u  - 2 c )/(2 cp), u v - u v gamma, u gamma - u], 
        2          2
   rho u  - 1    (c  rho - 1) u            u v     2     u
[- ----------, - --------------, 0, 0], [- ---, - c  v, ---, 0], 
         2            rho                  rho          rho
      rho
          2        3             2              2         3
[((rho u v  + rho u  - 2 u) gamma  + ((- rho u v ) - rho u  + 2 u) gamma
      2                 2                 2
 - 2 c  rho u)/(2 cp rho  gamma - 2 cp rho ), 
   2         2     2                 2      2       2
((c 

interrupt: Execution interrupted

Note that in Weiss and Smith, 

For the matrix K we still need to derive it, for now we just give the matrix:

In [ ]:
K:matrix([1,0,0,0],[-u,1,0,0],[-v,0,1,0],[-cp*T,0,0,1]);

In [ ]:
invert(K);

In [ ]:
Gnc:matrix([1/b^2,0,0,diff(rho,T)],[0,rho,0,0],[0,0,rho,0],[0,0,0,rho*cp]);

In [ ]:
invert(Gnc);

In [ ]:
eigenvalues(invert(Gnc));

In [ ]:
Atilde;

preconditioning matrix, which can then be derived from K.M:
IN the end, this is the matrix in fron of the temporal termf of the equations in conservation form with primitive variables:

In [ ]:
G:matrix([1/b^2,0,0,diff(rho,T)],[u/b^2,rho,0,diff(rho,T)*u],[v/b^2,0,rho,diff(rho,T)*v],[cp*T/b^2,0,0,rho*cp+diff(rho,T)*cp*T]);

In [ ]:
Ginv : ratsimp(invert(G));

In [ ]:
L:eigenvalues(subst(diff(rho,T)=1/(R*T),Ginv));
ratsimp(L);

In [ ]:
(ratsimp(Ginv.Atilde));

note that $p=(\gamma-1)*(E-\frac{1}{2}\rho u^2)$

In [ ]:
kill(E);
Ftilde : subst([E=p/(gamma-1)+rho*u*u/2],F);
Ftilde: transpose(subst([rho=U1,u=U2,p=U3],Ftilde));

In [ ]:
Utilde;

In [ ]:
Vtilde;

In [ ]:
jacobian(args(Ftilde)[1],args(Utilde)[1]);

We now try to go to directly to primitive variables (p,u,T)

In [ ]:
F;
Ftilde : subst([E=rho*H-p],F);
Ftilde : transpose(subst(H=cp*T+u*u/2,Ftilde));

now replace all densities with pressures:

In [ ]:
Ftilde : subst(rho=gamma*p/(c*c),Ftilde);

In [ ]:
V;

In [ ]:
dfdv : (jacobian(args(Ftilde)[1],args(V)[1]));

In [ ]:
eigenvalues(dfdv);

In [ ]:
Ginv;

In [ ]:
Ginv : subst(diff(rho,T)=-rho/T,Ginv);

In [ ]:
dfdv : subst(c^2=gamma*p/rho,dfdv);

In [ ]:
dfdv : subst(p=rho*R*T,dfdv);

In [ ]:
Gdfdv : ratsimp(Ginv.dfdv);

In [ ]:
eigenvalues(dfdv);

We now need to construct the matrix $\Gamma^{-1} \frac{\partial F}{\partial V}$

Note that this is simply the matrix in front of the convective flux.

With $\frac{\partial F}{\partial V} = \frac{\partial F}{\partial U} \cdot \frac{\partial U}{\partial V}$


In [ ]:
dFdV : ratsimp(dFdU*dUdV);
dFdV : ratsimp(subst(consSystem,dFdV));
dFdV : ratsimp(subst(E=rho*e+rho*(u*u+v*v)/2,dFdV));
ratsimp(eigenvalues(dFdV));
ratsimp(eigenvalues(ratsimp(Ginv*dFdV)));

H=cpT+(u^2+v^2+w^2)/2; 

so:
$$\frac{dH}{du}=u $$
$$\frac{dH}{dv}=v $$
$$\frac{dH}{dw}=w $$
$$\frac{dH}{dT}=cp $$


weiss and smith eq. 2

In [143]:
gradef(H,T,cp);
gradef(H,u,u);
gradef(H,v,v);
gradef(H,w,w);

(%o195)                                H

(%o196)                                H

(%o197)                                H

(%o198)                                H

In [144]:
dWdQ : ratsimp(jacobian(args(W)[1],args(Q)[1]));


args: argument must be a non-atomic expression; found W



 -- an error. To debug this try: debugmode(true);


In [ ]:
G : ratsubst(1/b^2,diff(rho,p),dWdQ);

In [ ]:
G : ratsimp(ratsubst(cp*T+u^2/2,H,G));

In [ ]:
Ginv : ratsimp(invert(G));

In [ ]:
B : ratsimp(Ginv.A1.G);

In [ ]:
eigenvalues(B);

In [ ]:
/*remove(rho,dependency);*/
dUdV : ratsimp(jacobian(args(U)[1],args(V)[1]));

In [ ]:
M : dUdV;
Minv : ratsimp(invert(M));
A1;
Atilde : ratsimp(Minv.A1.M);
eigenvalues(Atilde);

In [ ]:
M1 : ratsimp(ratsubst(cp*T+u^2/2,H,M));

In [ ]:
M1inv : ratsimp(invert(M1));

In [ ]:
A : ratsimp(u,U,A);
Atilde : A;

weiss and smith Eq.3

In [ ]:
/*K:matrix([1,0,0,0,0],[-u,1,0,0,0],[-v,0,1,0,0],[-w,0,0,1,0],[-(H-(u^2+v^2+w^2)),-u,-v,-w,1]);*/
K:matrix([1,0,0],[-u,1,0],[-(H-(u^2)),-u,1]);

weiss and smith eq.4

In [ ]:
Gp:ratsimp(K.dWdQ);

weiss and smith, eq. 10

In [ ]:
G : ratsimp(invert(K).Gp);

In [ ]:
ratsimp(jacobian(args(Ftilde)[1],args(Q)[1]));